In [1]:
import pandas as pd

In [2]:
lung = pd.read_csv('results/HumanGEM_Lung_COVID19_local2_1_3_2_fastcore_t0_msc_filtered.csv', index_col=0).drop('fba_simulation', axis=1).to_dict('records')
len(lung)

870

In [3]:
liver = pd.read_csv('results/HumanGEM_Liver_COVID19_local2_1_3_2_fastcore_t0_msc_filtered.csv', index_col=0).drop('fba_simulation', axis=1).to_dict('records')
len(liver)

731

In [4]:
heart = pd.read_csv('results/HumanGEM_Heart_COVID19_local2_1_3_2_fastcore_t0_msc_filtered.csv', index_col=0).drop('fba_simulation', axis=1).to_dict('records')
len(heart)

1991

In [5]:
kidney = pd.read_csv('results/HumanGEM_Kidney_COVID19_local2_1_3_2_fastcore_t0_msc_filtered.csv', index_col=0).drop('fba_simulation', axis=1).to_dict('records')
len(kidney)

716

In [6]:
intestine = pd.read_csv('results/HumanGEM_Intestine_COVID19_local2_1_3_2_fastcore_t0_msc_filtered.csv', index_col=0).drop('fba_simulation', axis=1).to_dict('records')
len(intestine)

1483

In [7]:
intersection = [x for x in lung if x in liver]
intersection = [x for x in intersection if x in heart]
intersection = [x for x in intersection if x in kidney]
intersection = [x for x in intersection if x in intestine]
len(intersection)


3

In [8]:
intersection

[{'Reaction 1': 'LNSTLSr', 'Reaction 2': 'HMR_1465'},
 {'Reaction 1': 'HMR_1467', 'Reaction 2': 'r0780'},
 {'Reaction 1': 'r0780', 'Reaction 2': 'HMR_1470'}]

In [9]:
import cobra

In [10]:
model = cobra.io.read_sbml_model('../models/HumanGEM_Consistent_COVID19.xml')
model

Name,HumanGEM
Memory address,190d8f02308
Number of metabolites,6149
Number of reactions,10347
Number of genes,2976
Number of groups,142
Objective expression,1.0*biomass_human - 1.0*biomass_human_reverse_fb2f2
Compartments,"Cytosol, Lysosome, Endoplasmic reticulum, Extracellular, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [11]:
gene_dataset = {}

i = 0

for cut_set in range(len(intersection)):
    d = {}
    
    gp_rule_1 = str(model.reactions.get_by_id(intersection[cut_set]['Reaction 1']).gpr).split(' or ')
    gp_rule_2 = str(model.reactions.get_by_id(intersection[cut_set]['Reaction 2']).gpr).split(' or ')
    
    if len(gp_rule_1) == 1 and len(gp_rule_2) == 1:
        gp_rule_1= gp_rule_1[0].split(' and ')
        gp_rule_2= gp_rule_2[0].split(' and ')
        
        if len(gp_rule_1) == 1 and len(gp_rule_2) == 1:
            d['Reaction 1'] = gp_rule_1[0]
            d['Reaction 2'] = gp_rule_2[0]
    
            gene_dataset[i] = d
            i += 1
            
gene_dataset

{0: {'Reaction 1': 'ENSG00000160285', 'Reaction 2': 'ENSG00000079459'},
 1: {'Reaction 1': 'ENSG00000079459', 'Reaction 2': 'ENSG00000149809'},
 2: {'Reaction 1': 'ENSG00000149809', 'Reaction 2': 'ENSG00000104549'}}

In [12]:
gene_dataset = {}

i = 0

for cut_set in range(len(intersection)):
    d = {}
    
    gp_rule_1 = str(model.reactions.get_by_id(intersection[cut_set]['Reaction 1']).gpr).split(' or ')
    gp_rule_2 = str(model.reactions.get_by_id(intersection[cut_set]['Reaction 2']).gpr).split(' or ')
    
    if len(gp_rule_1) == 1 and len(gp_rule_2) == 1:
        d['Reaction 1'] = gp_rule_1[0]
        d['Reaction 2'] = gp_rule_2[0]
        
        gene_dataset[i] = d
        i += 1
            
gene_dataset

{0: {'Reaction 1': 'ENSG00000160285', 'Reaction 2': 'ENSG00000079459'},
 1: {'Reaction 1': 'ENSG00000079459', 'Reaction 2': 'ENSG00000149809'},
 2: {'Reaction 1': 'ENSG00000149809', 'Reaction 2': 'ENSG00000104549'}}

In [13]:
gene_df = pd.DataFrame.from_dict(gene_dataset).T
gene_df

,Reaction 1,Reaction 2
0,ENSG00000160285,ENSG00000079459
1,ENSG00000079459,ENSG00000149809
2,ENSG00000149809,ENSG00000104549


In [14]:
gene_df.to_csv('results/gene_targets_Desai_GTEx.csv')